# Weaviate Setup

In this notebook we setup the Weaviate vector database.

## Imports and setup

- We import needed libraries.
- We setup credentials via environment variables.

In [7]:
import weaviate
import weaviate.classes as wvc
import os
import cohere

from dotenv import load_dotenv

load_dotenv()

cohere_api_key = os.getenv('COHERE_PROD')
co = cohere.Client(cohere_api_key)

wcd_url = os.getenv('WEAVIATE_URL')
wcd_api_key = os.getenv('WEAVIATE_API')

## Collection Creation

- Weaviate operates with collections, which are similar to tables.
- Each collection can contain objects, which are like table rows.
- Each collection can be set with a schema, defining:
  - Which embedding model to be used.
  - Which reranker model to be used.
  - Which generative model to be used.
  - Object properties and schema.
  - Selecting properties to be vectorized.

In [8]:
from weaviate.classes.config import Configure, Property, DataType

with weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,
    auth_credentials=wvc.init.Auth.api_key(wcd_api_key),
    headers={"X-Cohere-Api-Key": cohere_api_key},
    skip_init_checks=True,
) as client:

    client.collections.create(
        name="OpenITI",
        vectorizer_config=[
            Configure.NamedVectors.text2vec_cohere(
                name="cluster",
                source_properties=["text"],
                model="embed-multilingual-v3.0"
            )
        ],
        properties=[
            Property(name="text", data_type=DataType.TEXT),
            Property(name="page", data_type=DataType.TEXT, skip_vectorization=True),
            Property(name="volume", data_type=DataType.TEXT, skip_vectorization=True),
            Property(name="author", data_type=DataType.TEXT, skip_vectorization=True),
            Property(name="book", data_type=DataType.TEXT, skip_vectorization=True),
            Property(name="topic", data_type=DataType.TEXT, skip_vectorization=True)
        ],
        reranker_config=Configure.Reranker.cohere(model="rerank-multilingual-v3.0"),
        generative_config=Configure.Generative.cohere(model="command-r-plus"),
    )

# Data upload

- Weaviate automatically creates embeddings for uploaded objects, based on the collection setup and schema.
- We import an existing dataframe, which is saved from the clustering stage.
- Weaviate supports dynamic batch uploads, which determines on it's own the rate of upload.

In [11]:
import pandas as pd

df = pd.read_parquet('df_chunks_clust.parquet')

In [13]:
df

,text,start_index,end_index,token_count,page,volume,author,book,embeddings,cluster,is_representative,cluster_topic
0,بسم الله الرحمن الرحيم أحمد الله أولا حمدا كثي...,0,1535,500,P001,V01,محمد بن محمد الغزالي أبو حامد,إحياء علوم الدين,"[0.02581787109375, 0.06256103515625, -0.007503...",2,False,Wisdom
1,في الأسفار فاقتصرت فيه على ذكر طرف الحديث وصحا...,1214,2792,500,P001,V01,محمد بن محمد الغزالي أبو حامد,إحياء علوم الدين,"[0.03253173828125, 0.07135009765625, -0.020233...",2,False,Wisdom
2,العلم طمعا في نيل ما تعبده الله تعالى به من تز...,2490,4034,498,P001,V01,محمد بن محمد الغزالي أبو حامد,إحياء علوم الدين,"[0.00664520263671875, 0.049041748046875, -0.03...",2,True,Wisdom
3,وشبكة للحطام فأما علم طريق الآخرة وما درج عليه...,3745,5359,499,P002,V01,محمد بن محمد الغزالي أبو حامد,إحياء علوم الدين,"[0.0269012451171875, 0.056640625, -0.026824951...",2,False,Wisdom
4,الكسب وكتاب الحلال والحرام وكتاب آداب الصحبة و...,5050,6653,499,P002,V01,محمد بن محمد الغزالي أبو حامد,إحياء علوم الدين,"[0.020660400390625, 0.05975341796875, -0.02926...",0,False,Spirituality
...,...,...,...,...,...,...,...,...,...,...,...,...
3973,منها كما تخرج الحبة فى حميل السيل ألا ترونها ت...,5127397,5129086,500,P545,V04,محمد بن محمد الغزالي أبو حامد,إحياء علوم الدين,"[0.006313323974609375, 0.068603515625, -0.0333...",3,False,Faith
3974,صلى الله عليه وسلم فقال هم الذين لا يكتوون ولا...,5128730,5130355,499,P546,V04,محمد بن محمد الغزالي أبو حامد,إحياء علوم الدين,"[0.01000213623046875, 0.06353759765625, -0.032...",3,False,Faith
3975,فأعطانى هكذا وفرج عبد الله بن أبى بكر بين يديه...,5130037,5131693,500,P546,V04,محمد بن محمد الغزالي أبو حامد,إحياء علوم الدين,"[0.03472900390625, 0.058441162109375, -0.03292...",3,False,Faith
3976,مكانه النار يهوديا أو نصرانيا فاستحلفه عمر بن ...,5131347,5133042,498,P546,V04,محمد بن محمد الغزالي أبو حامد,إحياء علوم الدين,"[0.044952392578125, 0.06646728515625, -0.04867...",5,False,Afterlife


In [18]:
counter = 0
interval = 100

with weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,
    auth_credentials=wvc.init.Auth.api_key(wcd_api_key),
    headers={"X-Cohere-Api-Key": cohere_api_key},
    skip_init_checks=True
) as client:
    
    with client.batch.dynamic() as batch:

        for i, row in df.iterrows():
    
            properties = {
                "text": row['text'],
                "page": row['page'],
                "volume": row['volume'],
                "author": row['author'],
                "book": row['book'],
                "topic": row['cluster_topic']
            }

            batch.add_object(properties=properties,collection="OpenITI")
            
            counter += 1
            
            if counter % interval == 0:
                print(f"Imported {counter} articles...")

print(f"Finished importing {counter} articles.")

Imported 100 articles...
Imported 200 articles...
Imported 300 articles...
Imported 400 articles...
Imported 500 articles...
Imported 600 articles...
Imported 700 articles...
Imported 800 articles...
Imported 900 articles...
Imported 1000 articles...
Imported 1100 articles...
Imported 1200 articles...
Imported 1300 articles...
Imported 1400 articles...
Imported 1500 articles...
Imported 1600 articles...
Imported 1700 articles...
Imported 1800 articles...
Imported 1900 articles...
Imported 2000 articles...
Imported 2100 articles...
Imported 2200 articles...
Imported 2300 articles...
Imported 2400 articles...
Imported 2500 articles...
Imported 2600 articles...
Imported 2700 articles...
Imported 2800 articles...
Imported 2900 articles...
Imported 3000 articles...
Imported 3100 articles...
Imported 3200 articles...
Imported 3300 articles...
Imported 3400 articles...
Imported 3500 articles...
Imported 3600 articles...
Imported 3700 articles...
Imported 3800 articles...
Imported 3900 article

# Tests

- We connect and run a basic search query.
- We test for the correct structure of returned objects as defined in the schema.
- We check for the final size of the collection, at 3978 elements.

In [19]:
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,                                    # Replace with your Weaviate Cloud URL
    auth_credentials=wvc.init.Auth.api_key(wcd_api_key),    # Replace with your Weaviate Cloud key
    headers={"X-Cohere-Api-Key": cohere_api_key},           # Cohere API key
    skip_init_checks=True                                  # Skip gRPC and other startup checks
)

try:
    pass # Replace with your code. Close client gracefully in the finally block.
    questions = client.collections.get("OpenITI")

    response = questions.query.near_text(
        query="Wealth",
        limit=2
    )

    print(response.objects[0].properties)  # Inspect the first object

finally:
    client.close()  # Close client gracefully

{'volume': 'V04', 'text': 'إلا لذلك وليس لهم فى المال حظ إلا بقدر القوت فلا جرم اقتصروا على قدر القوت وما فضل فلم يمسكون بل أنفقوه فإن الإنفاق فيه الترياق وفى الإمساك السم ولو فتح للناس باب كسب المال ورغبوا فيه لمالوا إلى سم الإمساك ورغبوا عن ترياق الإنفاق فلذلك قبحت الأموال والمعنى به تقبيح إمساكها والحرص عليها للاستكثار منها والتوسع فى نعيمها بما يوجب الركون إلى الدنيا ولذتها فأما أخذها بقدر الكفاية وصرف الفاضل إلى الخيرات فليس بمذموم وحق كل مسافر أن لا يحمل إلا بقدر زاده فى السفر إذا صمم العزم على أن يختص بما يحمله فأما إذا سمحت نفسه بإطعام الطعام وتوسيع الزاد على الرفقاء فلا بأس بالاستكثار وقوله عليه الصلاة والسلام ليكن بلاغ أحدكم من الدنيا كزاد الراكب معناه لأنفسكم خاصة ولا فقد كان فيمن يروى هذا الحديث ويعمل به من يأخذ مائة ألف درهم فى موضع واحد ويفرقها فى موضعه وإلا يمسك منها حبة ولما ذكر رسول الله صلى الله عليه وسلم أن الأغنياء يدخلون الجنة بشدة إستأذنه عبد الرحمن بن عوف رضى الله عنه فى أن يخرج عن جميع ما يملكه فأذن له فنزل جبريل عليه السلام وقال مره بأن يطعم المسكين ويكسو العار

In [20]:
response.objects

[Object(uuid=_WeaviateUUIDInt('1d8f7468-add5-4869-8a38-180eff2522ce'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'volume': 'V04', 'text': 'إلا لذلك وليس لهم فى المال حظ إلا بقدر القوت فلا جرم اقتصروا على قدر القوت وما فضل فلم يمسكون بل أنفقوه فإن الإنفاق فيه الترياق وفى الإمساك السم ولو فتح للناس باب كسب المال ورغبوا فيه لمالوا إلى سم الإمساك ورغبوا عن ترياق الإنفاق فلذلك قبحت الأموال والمعنى به تقبيح إمساكها والحرص عليها للاستكثار منها والتوسع فى نعيمها بما يوجب الركون إلى الدنيا ولذتها فأما أخذها بقدر الكفاية وصرف الفاضل إلى الخيرات فليس بمذموم وحق كل مسافر أن لا يحمل إلا بقدر زاده فى السفر إذا صمم العزم على أن يختص بما يحمله فأما إذا سمحت نفسه بإطعام الطعام وتوسيع الزاد على الرفقاء فلا بأس بالاستكثار وقوله عليه الصلاة والسلام ليكن بلاغ أحدكم من الدنيا كزاد الراكب معناه لأنفسكم خاصة ولا فقد كان فيمن يروى هذا الحديث ويعمل به من يأخذ مائة ألف درهم 

In [22]:
book_collection = client.collections.get('OpenITI')

In [23]:
book_collection.aggregate.over_all()

AggregateReturn(properties={}, total_count=3978)

In [24]:
all_obj = [chunk for chunk in book_collection.iterator()]

In [25]:
len(all_obj)

3978